In [ ]:
# import json
# import boto3
# import yfinance as yf 
# import pandas as pd

# kinesis = boto3.client("kinesis", "us-east-2")

# def getReferrer(company):
#     start_date = '2021-11-30'
#     end_date = '2021-12-01'
#     interval = '5m'
#     download = yf.Ticker(company).history(start=start_date, end=end_date, interval=interval)
#     return download



# def lambda_handler(event, context):
#     companies = ['FB', 'SHOP', 'BYND', 'NFLX', 'PINS', 'SQ', 'TTD', 'OKTA', 'SNAP', 'DDOG']
#     for company in companies:
#          for index, rows in getReferrer(company).iterrows():
#              data = {}
#              data['high'] = rows.High
#              data['low'] = rows.Low
#              data['ts'] = str(index)
#              data['name'] = company
#              json_converter = json.dumps(data, separators=(',', ': '))+"\n"
#              print(json_converter)
#              kinesis.put_record(StreamName="sta9760stream1", Data=json_converter, PartitionKey = "partitionkey")
#     return {
# 	'statusCode': 200,
#         'body': json_converter
# 	}

In [ ]:
#######################GOOD CODE###############################

import json
import boto3
import yfinance as yf 
import pandas as pd

kinesis = boto3.client('kinesis', "us-east-2")

def lambda_handler(event, context):
    
    dates = []
    highs = []
    lows = []
    ticker = []
    stocks = ['FB', 'SHOP', 'BYND', 'NFLX', 'PINS', 'SQ', 'TTD', 'OKTA', 'SNAP', 'DDOG']
   
    stock_data = yf.download(stocks,
                          period="1d",
                          interval="5m",
                          start="2021-11-30",
                          end="2021-12-01",
                          group_by="ticker")
                          
    stock_data.reset_index(inplace=True)
    
    for stock in stocks:
        for i in range(len(stock_data)):
            dates.append(str(stock_data[['Datetime', stock]]['Datetime'].iloc[i]))
            highs.append(stock_data[['Datetime', stock]][stock]['High'].iloc[i])
            lows.append(stock_data[['Datetime', stock]][stock]['Low'].iloc[i])
            ticker.append(stock)

    main_data = pd.DataFrame(highs, columns=['high'])
    main_data['low'] = lows
    main_data['ts'] = dates
    main_data['name'] = ticker
    
    for i in range(len(main_data)):
        json_data = json.dumps(main_data.iloc[i].to_dict())+"\n"
        print(json_data)
        kinesis.put_record(
                StreamName="sta9760stream1",
                Data=json_data,
                PartitionKey="partitionkey")